In [1]:
from __future__ import print_function
CUDA_LAUNCH_BLOCKING=1

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.datasets import imdb
from keras import optimizers

import pandas as pd
import numpy as np
import os




import torch
from torch.autograd import Variable
from torch import optim,nn
from torch.utils.data import TensorDataset, DataLoader


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]



## Processing of new ninety file 

In [4]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [5]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [6]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [7]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'t', 'ň', 'ž', 'h', 'a', 'x', 'i', 'k', '7', 's', '2', 'é', 'ú', 'ů', 'z', 'l', '.', 'j', 'ý', 'č', 'e', 'n', 'p', 'š', '5', 'r', 'c', '0', 'ó', '6', 'ť', '1', 'í', 'q', ' ', '4', 'y', '3', '8', 'á', 'ř', 'v', 'ď', 'g', 'u', 'f', 'ě', 'b', 'd', 'o', 'END', '9', 'm', 'w'}
vocab length is  54
length of data_set is  15226


In [8]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [9]:
print(char_index)

{'ú': 12, 'ň': 1, 'ž': 2, 'h': 3, 'y': 36, 'a': 4, 'x': 5, 'k': 7, 't': 0, 's': 9, '2': 10, 'é': 11, 'ů': 13, 'ý': 18, 'l': 15, 'j': 17, 'z': 14, 'n': 21, 'š': 23, 'p': 22, '.': 16, 'r': 25, 'c': 26, '0': 27, 'ó': 28, '5': 24, 'ť': 30, '1': 31, 'č': 19, 'í': 32, ' ': 34, '4': 35, 'á': 39, 'i': 6, 'e': 20, '3': 37, '8': 38, '7': 8, 'ř': 40, 'ď': 42, 'g': 43, 'u': 44, 'd': 48, 'f': 45, 'ě': 46, 'b': 47, 'q': 33, 'o': 49, '6': 29, 'w': 53, 'END': 50, '9': 51, 'm': 52, 'v': 41}


In [10]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.9
train = data_set1[msk]
test = data_set1[~msk]     

msk = np.random.rand(len(czech)) < 0.9


vtrain = czech[msk]
vtest = czech[~msk]



In [11]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [12]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

# These are set of indian names, just for adding a little robustness and to construct that latent structure in names 

In [13]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_x = []
train_y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_x.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_y.append(1)
    else:
        train_y.append(0)
    
train_x=np.asarray(train_x)
train_y=np.asarray(train_y)

In [14]:
test_x = []
test_y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_x.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_y.append(1)
    else:
        test_y.append(0)
    
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

# These are Czech names, well, that's what the program is all about 

In [15]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in vtrain.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in vtrain.m_or_f:
    if i == 'm':
        vtrain_y.append(1)
    else:
        vtrain_y.append(0)
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

In [16]:
vtest_x = []
vtest_y = []

train_name = [str(i) for i in vtest.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtest_x.append(tmp)
for i in vtest.m_or_f:
    if i == 'm':
        vtest_y.append(1)
    else:
        vtest_y.append(0)
                       
vtest_x = np.asarray(vtest_x)
vtest_y = np.asarray(vtest_y)

# Combining all training set, and all test set 

In [17]:
print("These are all training data!")
print("Shape of indian training name:", train_x.shape)
print("Shape of Czech training name:", vtrain_x.shape)


These are all training data!
Shape of indian training name: (13738, 30, 54)
Shape of Czech training name: (1125, 30, 54)


In [18]:
print("Combining them all...")
net_train_x = np.concatenate([train_x, vtrain_x], axis = 0)
net_train_y = np.concatenate([train_y, vtrain_y], axis = 0)
print("Now net shape is:", net_train_x.shape)
print("Now they're combined, but we must shuffle them too...shuffling")
np.random.shuffle(net_train_x)
np.random.shuffle(net_train_y)
print("Shuffled!!")

Combining them all...
Now net shape is: (14863, 30, 54)
Now they're combined, but we must shuffle them too...shuffling
Shuffled!!


In [19]:
print("These are all test data!")
print("Shape of indian test name:", test_x.shape)
print("Shape of Czech test name:", vtest_x.shape)


These are all test data!
Shape of indian test name: (1488, 30, 54)
Shape of Czech test name: (125, 30, 54)


In [20]:
print("Combining them all...")
net_test_x = np.concatenate([test_x, vtest_x], axis = 0)
net_test_y = np.concatenate([test_y, vtest_y], axis = 0)
print("Now net shape is:", net_test_x.shape)
print("Now they're combined, but we must shuffle them too...shuffling")
np.random.shuffle(net_test_x)
np.random.shuffle(net_test_y)
print("Shuffled!!")

Combining them all...
Now net shape is: (1613, 30, 54)
Now they're combined, but we must shuffle them too...shuffling
Shuffled!!


### Making BatchLoader

In [21]:
#Creating Tensor Datasets
train_data = TensorDataset(torch.from_numpy(net_train_x).double(), torch.from_numpy(net_train_y).long())
test_data = TensorDataset(torch.from_numpy(net_test_x).double(), torch.from_numpy(net_test_y).long())

#Now creating Data Loaders
batch = 100

train_loader = DataLoader(train_data, shuffle = True, batch_size = batch)
test_loader = DataLoader(test_data, shuffle = True, batch_size = batch)

dataiter = iter(train_loader)

sample_x, sample_y = dataiter.next()

print("Sample input size of one instance from iter is: ", sample_x.size()) #Batch size, Sequence Length, Feature Dimension
print("Sample target size of that same one instance from iter is", sample_y.size())

Sample input size of one instance from iter is:  torch.Size([100, 30, 54])
Sample target size of that same one instance from iter is torch.Size([100])


## Model in PyTorch 

In [22]:
dtype = torch.DoubleTensor

class gender(nn.Module):
    def __init__(self, embedding_dim, no_units, output_dim, n_layers):
        super(gender, self).__init__()
        self.no_units = no_units
        self.n_layers = n_layers
        self.lstm = nn.LSTM(embedding_dim, no_units, n_layers, batch_first = True, bidirectional = True)
        self.linear = nn.Linear(no_units * 2, output_dim)
#         self.soft = nn.Softmax(dim = 1)
        
    def forward(self, x, hidden):
        hidden = self.init_hidden(x.size(0))
        lstmout, hidden = self.lstm(x.float(), hidden)
#         lstmout = lstmout.contiguous().view(-1, self.no_units)

        output = self.linear(lstmout)
        
#         output = self.soft(output)
        return output[:, -1, :], hidden
    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers * 2, batch_size, self.no_units).zero_(),
                      weight.new(self.n_layers * 2, batch_size, self.no_units).zero_())
        
        return hidden

In [23]:
mynet = gender(embedding_dim = 54, output_dim = 2, no_units = 50, n_layers = 2)

In [24]:
def trainn(net, batch_size = 100, epochs = 50, lr = 0.01, clip = 5, print_every = 10):
    global check
    net.train()
    net.float()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr = lr)
    
#     net()
    counter = 0
    val_losses = []
#     train_permutation = torch.randperm(trainx.shape[0])
#     test_permutation = torch.randperm(testx.shape[0])

    
    for i in range(epochs):
        
        #Initializing hidden state
        hd = net.init_hidden(batch_size)
        for batch_i, target_i in train_loader:
            

        
            inputs, targets = batch_i, target_i

            hd = tuple([each.data for each in hd])

            net.zero_grad()

            outputs, hd = net(inputs, hd)
            print(type(outputs))
            print(targets)
            loss = criterion(outputs, targets)
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            if i % 1 == 0:
                net.eval()
                val_h = net.init_hidden(batch_size)
                
                for batch_j, target_j in test_loader:
                    
                    vinputs, vtargets = batch_j, target_j

                    val_h = tuple([each.data for each in val_h])

                    output_h, val_h = net(vinputs, val_h)
                    val_loss = criterion(output_h, vtargets)
                    val_losses.append(val_loss.item())

            net.train()

            print("Epoch: {}/{} ...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.4f}...".format(loss.item()),
                  "Val Loss:{:.4f}".format(np.mean(val_losses)))

In [ ]:
trainn(mynet)

In [ ]:
net_train_x[indices].shape

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [ ]:
score, acc = model.evaluate(vtest_x, vtest_y)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
name=["riya"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

In [ ]:
pred

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [ ]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

In [ ]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

In [ ]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

In [ ]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [ ]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [ ]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [ ]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")